# Tracking with YOLO Detection and a Simple IOU Tracker

This notebook demonstrates how to use a YOLOv5 model for detecting cups in a sequence of images (one frame per image) and then apply a simple tracker to assign consistent IDs across frames. 

The notebook performs the following steps:

1. **Load YOLOv5:** The model is loaded using `torch.hub` (pretrained on COCO).
2. **Filter detections:** Only detections corresponding to the class "cup" are retained.
3. **Simple Tracking:** A basic tracker associates detections across frames using IoU matching.

Ensure that your images are placed in a folder called `frames` and that you have installed the required packages (e.g. `torch`, `opencv-python`).

In [4]:
import cv2
import os
import glob
import torch
import numpy as np

def compute_iou(box1, box2):
    """
    Computes the Intersection over Union (IoU) of two boxes.
    Box format: [x1, y1, x2, y2]
    """
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (box1[2] - box1[0]) * (box1[3] - box1[1])
    boxBArea = (box2[2] - box2[0]) * (box2[3] - box2[1])
    iou = interArea / float(boxAArea + boxBArea - interArea + 1e-5)
    return iou

class SimpleTracker:
    def __init__(self, iou_threshold=0.3):
        self.iou_threshold = iou_threshold
        self.next_id = 0
        self.tracks = {}  # dictionary mapping track_id -> bbox (format: [x1,y1,x2,y2])

    def update(self, detections):
        """
        Update tracks based on the new detections.
        `detections` is a list of bounding boxes in [x1,y1,x2,y2] format.
        Returns a list of tuples: (bbox, track_id).
        """
        new_tracks = {}
        assignments = []
        used_track_ids = set()

        for det in detections:
            best_iou = 0
            best_track_id = None
            for track_id, bbox in self.tracks.items():
                iou = compute_iou(det, bbox)
                if iou > best_iou and iou >= self.iou_threshold and track_id not in used_track_ids:
                    best_iou = iou
                    best_track_id = track_id
            if best_track_id is not None:
                new_tracks[best_track_id] = det
                assignments.append((det, best_track_id))
                used_track_ids.add(best_track_id)
            else:
                # create a new track
                new_id = self.next_id
                self.next_id += 1
                new_tracks[new_id] = det
                assignments.append((det, new_id))
        self.tracks = new_tracks
        return assignments

# Load YOLOv5 model (requires internet connection for the first time)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Get the class id for 'cup' from the model's names
cup_class_id = None
for k, v in model.names.items():
    if v == 'cup':
        cup_class_id = k
        break
if cup_class_id is None:
    print("Cup class not found in YOLO model.")

# Folder containing image frames
frames_folder = './TP3_data/frames'
if not os.path.isdir(frames_folder):
    raise Exception(f"Folder '{frames_folder}' not found. Please create it and add your image frames.")

# Get sorted list of image files
image_files = sorted(glob.glob(os.path.join(frames_folder, '*.jpg')))
if len(image_files) == 0:
    raise Exception(f"No images found in folder '{frames_folder}'.")

# Initialize our simple tracker
tracker = SimpleTracker(iou_threshold=0.3)

for image_path in image_files:
    frame = cv2.imread(image_path)
    if frame is None:
        print(f"Error reading image: {image_path}")
        continue

    # Run YOLO detection on the frame
    results = model(frame)
    detections = results.xyxy[0].cpu().numpy()  # each row: [x1, y1, x2, y2, confidence, class]

    # Filter detections for the cup class and a minimum confidence threshold
    cup_detections = []
    conf_threshold = 0.3
    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        if int(cls) == cup_class_id and conf >= conf_threshold:
            # Convert detection to integer format and add to list
            cup_detections.append([int(x1), int(y1), int(x2), int(y2)])

    # Update tracker with the current detections
    assignments = tracker.update(cup_detections)

    # Draw bounding boxes and IDs on the frame
    for bbox, track_id in assignments:
        x1, y1, x2, y2 = bbox
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)

    # Display the frame with detections and tracking
    cv2.imshow('YOLO + Tracker', frame)
    key = cv2.waitKey(30) & 0xFF
    if key == 27:  # Press Esc to exit
        break

cv2.destroyAllWindows()

Using cache found in C:\Users\war machine/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-3-19 Python-3.12.8 torch-2.5.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\war machine/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\war machine/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\war machine/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\war machine/.cache\t

KeyboardInterrupt: 

### Instructions

1. Place your image frames (e.g. JPG files) in a folder named `frames` (or change the folder path in the code).
2. Run this notebook. The YOLOv5 model will load (ensure you have internet access initially) and the notebook will process each frame: detecting cups and assigning them a consistent ID using the simple tracker.
3. Press `Esc` in the image display window to exit the loop.